# **PeNSE** - Pesquisa Nacional de Saúde do Escolar

![foto_escola_precaria.jpg](https://mediastorage.cnnbrasil.com.br/IMAGES/00/00/00/3461_D11F71BFD8C922EA.jpg)

> Fonte: CNN Brasil

## Apresentação

Olá, meu nome é Marcela Gomes Corrêa e atualmente sou servidora pública do executivo federal. Moro em Brasília e trabalho no Ministério da Economia. Vocês podem me encontrar nas seguintes redes sociais:

*   [LinkedIn](www.linkedin.com/in/marcelagomescorrea)
*   [Medium](https://marcelagomescorrea.medium.com/)
*   [GitHub](https://github.com/marcelagomescorrea)

Sou formada em Ciência da Computação pela Universidade de Brasília (UnB) e tenho especialização em Sistemas de Informação. Já fui programadora de sistemas web para eleições on line, mas atualmente trabalho com normativos relacionados à governança de dados. Minha experiência na área de dados começou pra valer quando entrei em um projeto de Big Data dentro do governo.

Apesar das limitações do meu atual cargo, sempre tive fascínio pela área de Data Science e, após participar da Imersão Dados da Alura, foi amor à primeira vista. Hoje, sou uma eterna estudante da área e escrevo algumas análises aqui e alguns artigos acolá.







# Introdução


---



#### **Sobre a Base**

A Pesquisa Nacional de Saúde do Escolar, ou **PeNSE**, é uma pesquisa realizada pelo IBGE com escolares adolescentes no Brasil, desde 2009, e é fruto de parceria com o Ministério da Saúde, tendo o apoio do Ministério da Educação.

Desde a sua primeira edição em 2009, a população-alvo da PeNSE são escolares do 9º ano do ensino fundamental (antiga 8ª série) das escolas públicas e privadas dos municípios e das capitais. Já na edição de 2015, optou-se por acrescentar uma outra amostra com escolares de 13 a 17 anos de idade frequentando as etapas do 6º ao 9º ano do ensino fundamental (antigas 5ª a 8ª séries) e da 1ª a 3ª série do ensino médio, e é com essa amostra que escolhi trabalhar.

#### **Motivação**

> Ninguém nasce mulher: torna-se mulher. ***O Segundo Sexo, Simone de Beauvoir***

Ícone do pensamento feminista, a filósofa e escritora, Simone de Beauvoir, defendia que o corpo biológico de uma fêmea (sexo, sentido biológico) torna-se mulher a partir de uma cultura, e não de regras até então tidas como naturais, ou seja, as diferenças sociais entre homens e mulheres não é algo natural, mas acidental, provocado pela sociedade.

Em Julho/2020, o Brasil se absteve em votação na ONU contra discriminação de mulheres e meninas, se alinhando a governos ultraconservadores em questões de gênero como Líbia, Congo e Afeganistão. Esse é só mais um dos diversos atos do nosso atual governo que enfraquecem cada vez mais as políticas públicas voltadas para mulheres no país.

Em um governo que estuda criar um programa para incentivar a 'abstinência' sexual em vez da **educação sexual**, muitas jovens brasileiras engravidam cedo e, em muitos casos, se vêem obrigadas a abortar em situações impróprias.

Na presente análise, vamos explorar quais as dificuldades enfrentadas pelas adolescentes e jovens brasileiras e abordar temas como aborto e gravidez precoce.


---


**Fonte**:
-  Folha: [Brasil se abstém em votação na ONU contra discriminação de mulheres e meninas](https://www1.folha.uol.com.br/mundo/2020/07/brasil-se-abstem-em-votacao-na-onu-contra-discriminacao-de-mulheres-e-meninas)


## Resumo


---



Com a nossa análise, partiremos dos dados da base PeNSE de 2015, sobre a saúde dos escolares, para analisar o cenário de muitas meninas e jovens durante o período escolar.

Utilizaremos informações gerais, como região geográfica, cor/raça, idade e tipo de escola (pública ou privada), além de dados sobre uso de cigarro, álcool e drogas, saúde e corpo, violência e abuso e, por fim, dados sobre saúde sexual e reprodutiva.

Mesmo que esse dados sejam relativamente obsoletos (aprox. 5 anos atrás), é possível utilizá-los como parâmetros para desenharmos um cenário para as adolescentes brasileiras.

Após o 'desenho' desse cenário, verificaremos como estão os números de óbitos por aborto e nascimentos por mães jovens nos últimos anos e tentaremos relacionar com as informações analisadas da base PeNSE.

# Bibliotecas importadas

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import cm

from  zipfile import ZipFile
import csv
import requests

In [2]:
!pip install plotly==4.12.0

In [3]:
# Importando Plotly
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots

In [4]:
from statsmodels.stats.weightstats import zconfint

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [5]:
# Função para imprimir mais de um dataframe lado a lado
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

# Tratando os dados

## Criando dicionário com perguntas e respostas

Para os dados da base PeNSE, vamos primeiro criar um dicionário com perguntas e respostas para facilitar na hora de especificar as informações que queremos analisar.

In [6]:
def isInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

In [7]:
# inicializando variáveis
dicio_pense = {}
codigo = ''
pergunta = ''
chave = ''                
respostas = {}

url = 'https://raw.githubusercontent.com/marcelagomescorrea/bootcamp_alura/main/Modulo_03/Dicionario_PENSE_Microdados_AMOSTRA2_filtrado.csv?raw=true'

with requests.Session() as s:
    download = s.get(url)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=';', )
    
    for row in cr:
        if not isInt(row[0]): # checando se é um nome de coluna            
            dicio_pense[chave
                        ] = {
                'codigo': codigo,
                'pergunta': pergunta,
                'respostas': respostas
            }

            # adicionou as infos da coluna ao dicionário,
            # reinicializa as variáveis            
            codigo = row[0].strip()
            pergunta = row[1].strip()
            chave = row[2].strip()
            respostas = {}

        else:
            respostas[int(row[0].strip())] = row[1].strip()

In [8]:
dicio_pense['cor_raca']

{'codigo': 'VB01002',
 'pergunta': 'Qual é a sua cor ou raça?',
 'respostas': {1: 'Branca',
  2: 'Preta',
  3: 'Amarela',
  4: 'Parda',
  5: 'Indígena',
  99: 'Não informado'}}

In [9]:
dicio_pense['ja_estupro']

{'codigo': 'VB09016',
 'pergunta': 'Alguma vez na vida você foi forçado a ter relação sexual?',
 'respostas': {-1: 'Pulo no questionário',
  1: 'Sim',
  2: 'Não',
  99: 'Não informado'}}

## Importando os dados

Após criar o dicionário, vamos importar os dados de fato da nossa amostra 2, que considera estudantes de diferentes faixas etárias.

In [10]:
path = 'https://github.com/marcelagomescorrea/bootcamp_alura/blob/main/Modulo_03/datasets/PeNSE_2015_AMOSTRA2.zip?raw=true'
dados = pd.read_csv(path, sep=';', thousands = ".", decimal = ",", compression='zip')
dados.head()

,ANOPESQ,PAIS,REGEOGR,VB00004,VB01001,VB01002,VB01003,VB01004,VB01005,VB01006,VB01007,VB01008A,VB01010A,VB01011,VB01012,VB01013,VB01014,VB01015A,VB01016,VB01017,VB01018,VB01019,VB01020A,VB01021,VB01022,VB01023,VB01024,VB01025,VB01026,VB02001,VB02002,VB02004A,VB02010,VB02011,VB02013,VB02017A,VB02018A,VB02019A,VB02020A,VB02021,...,VB11005,VB11006,VB11007,VB12001,VB12002,VB12003,VB13001,VB13002A,VB13004A,VB13005,VB13006,VB13007,VB13008,VB13009,VB14001,VB14002,VB16001A01,VB16001A02,VB16001A03,VB16001A04,VB16001A05,VB16001A06,VB16001A07,VB16001A08,VB17001,VB17002,VB17003,VB17004,VB17005,VB17006,ESTRATO_EXP,ESTRATOGEOREG,PESO,V0006,V0007,V0008,V0041,aluno,escola,turma
0,2015,76,1,1,2,1,13,7,7,1,2,7,3,2,-1,1,1,1,1,1,2,4,2,4,1,2,2,5,1,8,3,4,3,6,6,1,6,1,-1,2,...,2,1,2,1,1,4,2,-1,3,1,1,-1,1,1,2,2,1,2,2,2,2,2,2,2,371.0,1570.0,37.1,157.0,2,1,1223,1,299.735235,1,2,4,1,1,1,296
1,2015,76,1,1,2,1,14,4,7,1,1,7,3,2,-1,1,1,1,1,1,2,2,2,4,1,2,2,5,3,8,2,1,8,8,6,1,1,5,-1,2,...,2,3,2,4,4,4,1,2,3,3,1,-1,1,1,2,2,2,2,2,1,2,2,2,2,502.0,1520.0,50.2,152.0,2,2,1223,1,355.170081,1,2,4,1,2,1,296
2,2015,76,1,1,1,4,13,6,7,1,1,5,5,2,-1,2,1,1,1,1,2,3,1,4,1,2,2,5,3,2,3,1,8,2,8,1,1,5,5,1,...,2,1,2,1,2,3,1,2,1,2,5,99,1,-1,2,2,1,1,1,1,1,1,1,1,482.0,1614.0,48.2,161.4,2,2,1223,1,299.735235,1,2,4,1,3,1,296
3,2015,76,1,1,1,1,14,6,7,1,1,7,4,2,-1,1,1,1,1,1,2,4,2,4,1,2,2,6,3,2,4,1,3,2,7,1,5,1,-1,2,...,2,1,2,2,2,4,1,10,3,1,2,-1,1,-1,2,2,1,2,2,2,1,2,2,2,694.0,1725.0,69.4,172.5,2,3,1223,1,355.170081,1,2,4,1,4,1,296
4,2015,76,1,1,1,1,13,9,7,1,2,5,4,2,-1,2,1,1,1,1,2,5,1,4,1,2,2,6,5,6,3,5,2,6,2,1,5,1,5,1,...,2,1,4,2,2,4,1,3,2,2,2,13,1,-1,1,2,1,1,1,1,1,1,1,1,789.0,1675.0,78.9,167.5,2,4,1223,1,299.735235,1,2,4,1,5,1,296


In [11]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16556 entries, 0 to 16555
Columns: 181 entries, ANOPESQ to turma
dtypes: float64(10), int64(171)
memory usage: 22.9 MB


## Adicionando e traduzindo colunas

**Função**

Inicialmente, vamos criar uma função que utilize o dicionário criado anteriormente para traduzir o nome e os valores de uma coluna usando o seu código como referência.

In [12]:
def traduzir_coluna(coluna_nome, dataset):
    coluna = dicio_pense[coluna_nome]
    dataset[coluna_nome] = dataset[coluna['codigo']].map(coluna['respostas'])

Agora que criamos a função de tradução, vamos adicionar e 'traduzir' algumas colunas antes de começarmos nossas análises.

**Região**

In [13]:
traduzir_coluna('regiao', dados)
dados[['REGEOGR', 'regiao']].sample(10)

,REGEOGR,regiao
337,1,Norte
9185,3,Sudeste
12405,4,Sul
11444,4,Sul
9131,3,Sudeste
14557,5,Centro-Oeste
14973,5,Centro-Oeste
3521,2,Nordeste
771,1,Norte
9114,3,Sudeste


**Série e Nível de Ensino**

In [14]:
coluna_serie = dicio_pense['serie']['codigo']

traduzir_coluna('serie', dados)

dados['serie'] = dados['serie'].str.replace(' do Ensino Fundamental','')
dados['serie'] = dados['serie'].str.replace(' Ensino Médio','')
dados['serie'] = dados['serie'].str.replace(' / ','/')

dados[[coluna_serie, 'serie']].sample(5)

,VB01021,serie
9908,2,7º ano/6ª série
9090,4,9º ano/8ª série
13492,3,8º ano/7ª série
9502,1,6º ano/5ª série
13682,3,8º ano/7ª série


In [15]:
dados['serie'].unique()

array(['9º ano/8ª série', '8º ano/7ª série', '2º ano', '6º ano/5ª série',
       '3º ano', '7º ano/6ª série', '1º ano', 'Não informado'],
      dtype=object)

In [16]:
dicio_ensino = {
    1: 'Ensino Fundamental',
    2: 'Ensino Fundamental',
    3: 'Ensino Fundamental',
    4: 'Ensino Fundamental',

    5: 'Ensino Médio',
    6: 'Ensino Médio',
    7: 'Ensino Médio',

    99: 'Não Informado'
}

dados['ensino'] = dados[coluna_serie].map(dicio_ensino)
dados[['ensino', coluna_serie]].sample(5)

,ensino,VB01021
14680,Ensino Médio,5
13173,Ensino Fundamental,2
16261,Ensino Fundamental,3
16470,Ensino Médio,6
10639,Ensino Fundamental,2


**Tipos de Escola**

In [17]:
traduzir_coluna('tipo_escola', dados)
dados[['tipo_escola', 'V0007']].sample(5)

,tipo_escola,V0007
7796,Pública,1
3681,Pública,1
6615,Privada,2
10662,Pública,1
7251,Privada,2


In [18]:
traduzir_coluna('situacao_escola', dados)
dados[['situacao_escola', 'V0006']].sample(5)

,situacao_escola,V0006
11130,Urbana,1
11177,Urbana,1
4833,Urbana,1
1338,Urbana,1
10394,Urbana,1


**IMC**

In [19]:
dados['IMC'] = dados["VB17003"] / ((dados["VB17004"]/100)**2)
dados['IMC']

0        15.051321
1        21.727839
2        18.502907
3        23.322831
4        28.122076
           ...    
16551    20.113223
16552    17.930782
16553    23.968489
16554    18.995343
16555    23.303246
Name: IMC, Length: 16556, dtype: float64

In [20]:
classes = [ 0, 18.5, 25, 30, 35, 40, 99]
labels = ['abaixo', 'normal', 'sobrepeso', 'obesidade grau I',
          'obesidade grau II', 'obesidade grau III']

dados['classe_IMC'] = pd.cut(x = dados['IMC'],
       bins = classes,
       labels = labels,
       include_lowest = True)

dados[['IMC', 'classe_IMC']].sample(5)

,IMC,classe_IMC
10050,26.484357,sobrepeso
11468,18.660182,normal
1908,23.689575,normal
495,22.386379,normal
15954,23.595456,normal


In [21]:
dados.query('IMC > 30')[['IMC', 'classe_IMC']]

,IMC,classe_IMC
15,34.506506,obesidade grau I
44,37.015532,obesidade grau II
66,31.473861,obesidade grau I
104,35.055632,obesidade grau II
110,31.054687,obesidade grau I
...,...,...
16451,32.171202,obesidade grau I
16503,36.634860,obesidade grau II
16533,32.506673,obesidade grau I
16536,30.231284,obesidade grau I


**Sexo**

In [22]:
traduzir_coluna('sexo', dados)
dados[['VB01001', 'sexo']].sample(5)

,VB01001,sexo
13599,2,Feminino
14415,2,Feminino
12965,2,Feminino
6338,1,Masculino
16006,1,Masculino


In [23]:
dados['sexo'].unique()

array(['Feminino', 'Masculino'], dtype=object)

**Cor/Raça**

In [24]:
traduzir_coluna('cor_raca', dados)
dados[['VB01002', 'cor_raca']].sample(5)

,VB01002,cor_raca
189,4,Parda
7096,5,Indígena
299,1,Branca
9151,1,Branca
6856,2,Preta


**Idade**

In [25]:
traduzir_coluna('idade', dados)
dados[['VB01003', 'idade']].sample(5)

,VB01003,idade
15427,13,13 anos
8765,13,13 anos
4828,11,11 anos ou menos
3206,16,16 anos
1795,16,16 anos


# Conhecendo nossa amostra

Tudo pronto: dicionário e dados importados corretamente. Agora vamos explorar alguns dados da nossa amostra, começando com informações mais gerais.

## Informações Gerais

### Região e Cor/Raça

In [26]:
df = dados.query('sexo == "Feminino"').groupby(['regiao', 'cor_raca'])['sexo'].agg(['count'])
df = df.reset_index()
df.columns = ['Região', 'Cor/Raça', 'Quantidade']
df.head()

,Região,Cor/Raça,Quantidade
0,Centro-Oeste,Amarela,96
1,Centro-Oeste,Branca,618
2,Centro-Oeste,Indígena,46
3,Centro-Oeste,Não informado,3
4,Centro-Oeste,Parda,799


In [27]:
fig = px.treemap(df, path = ['Região', 'Cor/Raça'], 
                 values = 'Quantidade', color='Quantidade',
                 color_continuous_scale='Purples')

fig.data[0].textinfo = 'label+text+value'

fig.update_layout(
    title_text = 'PeNSE - Pesquisa Nacional de Saúde do Escolar<br />'\
                 '<b>Perfil das Escolares (Região e Cor/Raça)</b>',
    template = 'plotly_white',
    autosize = False,
    width = 800,
    height = 700    
)

fig.show()

Pelo gráfico acima, podemos ver que a maioria das regiões apresenta proporções semelhantes na quantidade de escolares, com a seguinte ordem decrescente: parda, branca, preta, amarela e indígena.

Apenas duas regiões não apresentam esse comportamento: Sudeste, que possui números praticamente iguais para pardas e brancas, e a região Sul, que possui como maioria esmagadora as brancas.

### Faixa Etária e Cor/Raça

In [28]:
df2 = dados.query('sexo == "Feminino"').copy().groupby(['cor_raca', 'idade'])['sexo'].agg(['count'])
df2 = df2.query('cor_raca != "Não informado"').reset_index()
df2.head()

,cor_raca,idade,count
0,Amarela,11 anos ou menos,42
1,Amarela,12 anos,80
2,Amarela,13 anos,55
3,Amarela,14 anos,62
4,Amarela,15 anos,64


In [29]:
fig = px.bar(df2, x='idade', y='count', color='cor_raca', text='count')

fig.update_layout(
    title_text = 'PeNSE - Pesquisa Nacional de Saúde do Escolar: '\
                 '<b>Perfil das Escolares (Idade e Cor/Raça)</b>',
    template = 'plotly_white',
    autosize = False,
    width = 850,
    height = 600,

    xaxis_title_text = 'Faixa Etária',
    yaxis_title_text = 'Quantidade de Escolares',
    legend = dict(title='', orientation="h", y=0.959,
                  yanchor="bottom", xanchor="right", x = 1), 
)

fig.show()

No gráfico de barras acima, percebe-se proporções de cor/raça semelhantes para todas as faixas etárias, com a maioria sendo parda, seguida de branca. Também verifica-se, que a maioria das escolares encontra-se na faixa de 12 anos, seguida da faixa de 13 anos.

### Série e Idade por Tipo de Escola

Para analisar a relação entre série, idade e tipo de escola (pública/privada), decidi utilizar uma tabela com mapa de calor, para ver onde encontra-se a maioria das escolares.

In [30]:
df3 =  dados.copy().query('sexo == "Feminino"')
dados_escola_pub = df3.query(f"{dicio_pense['tipo_escola']['codigo']} == 1")
dados_escola_priv = df3.query(f"{dicio_pense['tipo_escola']['codigo']} == 2")

In [31]:
def destaca_valor(x, valor, cor_texto, cor_fundo):
    if type(valor) == str:
        is_true = np.array(x).astype(str) == valor
    else:
        is_true = np.array(x) == valor
    return [f'background-color: {cor_fundo}; color: {cor_texto}' if v else '' for v in is_true]

In [32]:
coluna_idade = dicio_pense['idade']['codigo']

# cruzamento de Série e Idade
escola_privada = pd.crosstab(dados_escola_priv['serie'], dados_escola_priv[coluna_idade],
                           rownames=['Série/Ano'],
                           colnames=['Idade']
                           ).reindex(['6º ano/5ª série', '7º ano/6ª série',
                                     '8º ano/7ª série', '9º ano/8ª série',
                                     '1º ano', '2º ano', '3º ano',
                                     'Não informado'])\
                        .style.set_table_attributes('class="dataframe"')\
                        .background_gradient(cmap='Oranges', axis=None)\
                        .apply(destaca_valor, valor=0,
                               cor_texto='white', cor_fundo='white')

escola_publica = pd.crosstab(dados_escola_pub['serie'], dados_escola_pub[coluna_idade],
                           rownames=['Série/Ano'],
                           colnames=['Idade']
                           ).reindex(['6º ano/5ª série', '7º ano/6ª série',
                                     '8º ano/7ª série', '9º ano/8ª série',
                                     '1º ano', '2º ano', '3º ano',
                                     'Não informado'])\
                        .style.set_table_attributes('class="dataframe"')\
                        .background_gradient(cmap='Purples', axis=None)\
                        .apply(destaca_valor, valor=0,
                               cor_texto='white', cor_fundo='white')

print('Idade vs Série por Tipo de Escola')
display('escola_privada', 'escola_publica')

Idade vs Série por Tipo de Escola


Idade,11,12,13,14,15,16,17,18,19
Série/Ano,,,,,,,,,
6º ano/5ª série,292,24,6,1,0,0,0,0,0
7º ano/6ª série,104,282,30,9,1,0,1,0,0
8º ano/7ª série,3,86,207,20,7,1,0,0,0
9º ano/8ª série,0,3,72,110,8,1,0,0,0
1º ano,0,0,2,94,155,31,6,0,1
2º ano,0,0,0,3,135,209,23,5,0
3º ano,0,0,0,0,4,64,105,11,2
Não informado,0,1,0,0,0,0,0,0,0
Idade,11,12,13,14,15,16,17,18,19


Naturalmente, a diagonal que possui a maior parte das escolares (cores mais fortes) contém a idade  para a série/ano correspondente. Abaixo dessa diagonal encontra-se as escolares *adiantadas*, ou seja, que estão em um grau acima do que deveria estar de acordo com a sua idade.

A mesma lógica pode ser utilizada para a região acima das diagonais, que possuem as escolares *atrasdas* para a sua série/ano. Sendo assim, claramente é verificado que para escolas públicas, há um maior número de 'repetentes'.

Tais fatos indicam algo que já é sabido: geralmente, há uma diferença na qualidade do ensino em escolares particulares e em escolas públicas. Aí podemos levantar diversos fatores como possíveis causas para essa diferença como: falta de investimento do governo na educação pública, seja na infraestrutura, remuneração de professores e etc, e desigualdade social vivenciada pelos alunos de baixa renda.

## Cigarro, Álcool e Drogas

In [33]:
cols = ['idade_bebeu_primeira_vez',
        'idade_fumou_primeira_vez',
        'idade_droga_primeira_vez']

dados_filtrados = dados.copy()
for col in cols:
    cod_col = dicio_pense[col]['codigo']
    dados_filtrados = dados_filtrados.query(f'{cod_col} != -1 & {cod_col} != 99 & {cod_col} != 1')
    
traduzir_coluna('idade_bebeu_primeira_vez', dados_filtrados)
traduzir_coluna('idade_fumou_primeira_vez', dados_filtrados)
traduzir_coluna('idade_droga_primeira_vez', dados_filtrados)

mulheres = dados_filtrados.query('sexo == "Feminino"')
       
# Para mulheres
df4 = pd.DataFrame([mulheres[c].value_counts() for c in cols]).T.copy()
df4.columns = ['Álcool', 'Cigarro', 'Drogas']

df4 = df4.reindex(['7 anos ou menos', '8 anos', '9 anos', '10 anos', '11 anos',
             '12 anos', '13 anos', '14 anos', '15 anos', '16 anos',
             '17 anos', '18 anos ou mais'])

print('Idade que experimentou pela primeira vez')
df4.style.set_table_attributes('class="dataframe"')\
        .background_gradient(cmap='Oranges', axis=0)

Idade que experimentou pela primeira vez


,Álcool,Cigarro,Drogas
7 anos ou menos,20,12,3
8 anos,10,5,1
9 anos,14,10,1
10 anos,31,26,4
11 anos,32,21,15
12 anos,97,70,58
13 anos,116,107,98
14 anos,95,103,101
15 anos,72,102,121
16 anos,23,45,75


Outro assunto importante a ser analisado são as idades que as adolescentes brasileiras experimentaram álcool, cigarro e droga. No mapa de calor acima, conseguimos observar que enquanto para **álcool** e o **cigarro** a maioria experimentou aos 13 anos, a maior parte teve sua primeira experiência com **drogas** aos 15 anos. Em qualquer um dos casos, essas idades evidenciam algo preocupante: as jovens começam a experimentar cigarro, álcool ou drogas muito cedo, o que, a posteri, pode levar a queda do rendimento escolar ou até a evasão escolar.

## Saúde e Corpo

```
satisfacao = VB11007
como considera o próprio corpo = VB11001
IMC
cor/raça = VB01002

```

In [34]:
traduzir_coluna('satisfacao_corpo', dados)
dados[['satisfacao_corpo', 'VB11007']].sample(5)

,satisfacao_corpo,VB11007
8387,Indiferente,3
15298,Não informado,99
16402,Insatisfeito(a),4
15446,Satisfeito(a),2
15193,Satisfeito(a),2


In [35]:
df5 = dados.query('sexo == "Feminino" and satisfacao_corpo != "Não informado" '\
                       'and cor_raca != "Não informado"')

In [36]:
cores = ['darkorange', 'firebrick', 'purple', 'forestgreen', 'dodgerblue']

# Gráfico
fig = px.box(df5, x='satisfacao_corpo', y='IMC', color='cor_raca',
             points='all', color_discrete_sequence=cores)

### ACRESCENTANDO AS FAIXAS DE CLASSES DE IMC
# Abaixo do Peso
fig.add_hrect(y0=0, y1=18.5, line_width=0, fillcolor="lightsalmon", opacity=0.3,
                layer="below")
fig.add_annotation(x=-1, y=15.5, text="Abaixo do peso", showarrow=False,
                   yshift=10, font=dict(color='salmon'))
# Normal
fig.add_hrect(y0=18.5, y1=25, line_width=0, fillcolor="silver", opacity=0.3,
                layer="below")
fig.add_annotation(x=-1, y=22, text="Normal", showarrow=False,
                   yshift=10, font=dict(color='grey'))
# Sobrepeso
fig.add_hrect(y0=25, y1=30, line_width=0, fillcolor="bisque", opacity=0.3,
                layer="below")
fig.add_annotation(x=-1, y=27, text="Sobrepeso", showarrow=False,
                   yshift=10, font=dict(color='orange'))
# Obesidade
fig.add_hrect(y0=30, y1=60, line_width=0, fillcolor="lightcoral", opacity=0.3,
                layer="below")
fig.add_annotation(x=-1, y=57, text="Obesidade", showarrow=False,
                   yshift=10, font=dict(color='firebrick'))

# Ajustes na imagem
fig.update_layout(
            title = 'PeNSE - Pesquisa Nacional de Saúde do Escolar: '\
                        '<b>IMC e Satisfação com o próprio corpo</b>',
            title_font = dict(size=18),
            template = 'plotly_white',     

            yaxis_title_text = 'IMC',
            legend = dict(title='', orientation="h", y=0.999,
                            yanchor="bottom", xanchor="right", x = 1),

            xaxis_title_text = 'Nível de Satisfação',
            xaxis_categoryorder = 'array',
            xaxis_categoryarray =  ['Muito insatisfeito(a)', 'Insatisfeito(a)',
                                    'Indiferente', 'Satisfeito(a)', 'Muito satisfeito(a)'],
)

fig.show()

    Pontos interessantes

- p/ o nível **Muito satisfeito(a)**, os dados são menos dispersos e próximos da faixa com IMC abaixo do peso --> padrão de beleza no Brasil: ser magra.
- a medida que sobe o nível de satisfação, menos dispersos são os dados.

Agora vamos analisar assuntos mais específicos, começando pelo tema saúde e corpo. Para essa análise, optei por relacionar o IMC, o nível de satisfação e a cor/raça das jovens escolares.

Pelos boxplots acima, percebemos inicialmente que a grande maioria dos IMCs, independente da cor/raça e do nível de satisfação, encontra-se na faixa *normal* (cinza). Além disso, olhando mais atentamente, vemos que à medida que aumenta o nível de satisfação, menos dispersos são os dados.

Por fim, o fato mais curioso é verificado ao analisar os níveis de satisfação que demonstram estar satisfeitas com o próprio corpo. Independente de cor/raça, há praticamente a mesma faixa de IMC e ela é muito próxima do limite superior o intervalo para **abaixo do peso**. Isso pode ser um triste indicativo de que as adolescentes enxergam um padrão de perfeição em corpos mais magros, um arquétipo imposto pela **cultura da magreza**. 

Tantas meninas e jovens (e até mulheres) acreditam que serão mais felizes se forem magras, mas na realidade diversos distúrbios alimentares se escondem atrás desse falso padrão de beleza imposto pela nossa sociedade.


---


**Fonte**:

[Drauzio Varella - O preço da magreza](https://drauziovarella.uol.com.br/para-as-mulheres/o-preco-da-magreza/)

## Saúde Sexual e Reprodutiva

```

- gravidez:
    já engravidou (VB08013)
    média de idade
```

In [37]:
traduzir_coluna('ja_engravidou', dados)
dados[['ja_engravidou', 'VB08013']].sample(5)

,ja_engravidou,VB08013
15121,Pulo no questionário,-1
6335,Pulo no questionário,-1
12031,Pulo no questionário,-1
7635,Pulo no questionário,-1
10703,Pulo no questionário,-1


In [38]:
df7 = dados.copy().query('sexo == "Feminino" & ja_engravidou == "Sim" & '\
                         f'{coluna_idade} != 11 & {coluna_idade} != 19')
racas = df7['cor_raca'].unique()

to_tabulate = []
for raca in racas:
    df7_filtrado = df7.query(f'cor_raca == "{raca}"')
    ic = zconfint(df7_filtrado[coluna_idade])
    to_tabulate.append([raca, ic[0].round(2), ic[1].round(2)])


In [39]:
from tabulate import tabulate
print('Média de idade das escolares que já engravidaram')
print(tabulate(to_tabulate, ["Cor/Raça", "IC ini", "IC fim"], tablefmt="fancy_grid"))

Média de idade das escolares que já engravidaram
╒════════════╤══════════╤══════════╕
│ Cor/Raça   │   IC ini │   IC fim │
╞════════════╪══════════╪══════════╡
│ Amarela    │    15.51 │    17.29 │
├────────────┼──────────┼──────────┤
│ Parda      │    15.96 │    16.59 │
├────────────┼──────────┼──────────┤
│ Branca     │    15.88 │    16.92 │
├────────────┼──────────┼──────────┤
│ Preta      │    15.1  │    16.44 │
├────────────┼──────────┼──────────┤
│ Indígena   │    13    │    13    │
╘════════════╧══════════╧══════════╛


Para esse terceiro bloco, abordamos Saúde Sexual e Reprodutiva. O foco foi a média de idade das adolescentes que já engravidaram, para isso fiz o filtro dos dados para escolares do sexo feminino que informaram já ter engravidado e separei de acordo com a raça/cor.

Como não temos a informação da idade de fato que as escolares engravidaram, a ideia aqui é tentar verificar a média da idade dessas jovens que já engravidaram anteriormente, seja qual for a idade. Para gerar a tabela acima, calculei o intervalo de confiança da média de idade dessas jovens para cada cor/raça, com nível de significância de 0,05.


---

Exceto para cor/raça indígena, os intervalos se mostram sutilmente semelhantes. Para a cor/raça indígena, vemos que a média mostrada para o intervalo de confiança calculado é menor do que para as outras cores/raças. Mas porque escolares indígenas engravidam mais cedo?

Fiz uma breve pesquisa sobre o tema e listarei possíveis motivos para esse fato. O primeiro deles é que os povos indígenas encaram a sexualidade de maneira muito mais natural do que outros povos, não possuindo a mesma influência exercida pela moral cristã, onde sexo é visto como algo pecaminoso.

Além disso, os povos nativos sempre iniciaram a vida sexual cedo, mas possuíam métodos naturais para controle da natalidade, mas que foram abandonados ou perdidos ao longo dos anos.

Por fim, adolescentes indígenas possuem um marco muito bem definido na sua vida fértil: a menstruação. Após esse marco, os indígenas acreditam que essas jovens deixam de ser crianças, se tornando mulheres e, portanto, já podem se reproduzir.



---


**Fontes**:


[Folha de São Paulo](https://www1.folha.uol.com.br/fsp/cotidian/ff31089810.htm) - Norte tem mais grávidas adolescentes

## Violência e Abuso

```
- abuso sexual:
    já sofreu estupro (VB09016)
    por quem
        namorado (VB0901701)
        amigo (VB0901702)
        responsável (VB0901703)
        familiar (VB0901704)
        desconhecido (VB0901705)
        outros (VB0901706)
```

In [40]:
cols_estupro = ['ja_estupro', 'estupro_namorado', 'estupro_amigo',
                'estupro_responsavel', 'estupro_familiar',
                'estupro_desconhecido', 'estupro_outros']

for col in cols_estupro:
    traduzir_coluna(col, dados)

In [41]:
df6 = dados.copy().query('sexo == "Feminino" & ja_estupro == "Sim"')
cols_estupro.remove('ja_estupro')
for col in cols_estupro:
    df6 = df6.query(f'{col} != "Pulo no questionário" & '\
                    f'{col} != "Não informado"')

In [42]:
df6 = pd.DataFrame(df6[cols_estupro].value_counts())
df6.columns = ['Total']
df6_T = df6.T.copy()
df6_T.columns = df6_T.columns.reorder_levels(['estupro_namorado', 'estupro_familiar',
                              'estupro_desconhecido', 'estupro_amigo',
                              'estupro_responsavel', 'estupro_outros'])

In [43]:
def magnify():
    propriedades = [('background-color', "red"),
                    ('color', "white")]

    return [    # primeira linha - namorado
                dict(selector="th.level0.col0",     props=propriedades),
                dict(selector="th.level0.col8",     props=propriedades),
                dict(selector="th.level0.col10",    props=propriedades),
                # segunda linha - familiar
                dict(selector="th.level1.col1",     props=propriedades),
                dict(selector="th.level1.col8",     props=propriedades),
                dict(selector="th.level1.col12",    props=propriedades),
                dict(selector="th.level1.col15",    props=propriedades),
                # segunda linha - desconhecido
                dict(selector="th.level2.col2",     props=propriedades),
                dict(selector="th.level2.col6",     props=propriedades),
                dict(selector="th.level2.col9",     props=propriedades),
                # terceira linha - amigo
                dict(selector="th.level3.col3",     props=propriedades),
                dict(selector="th.level3.col7",     props=propriedades),
                dict(selector="th.level3.col9",    props=propriedades),
                dict(selector="th.level3.col10",    props=propriedades),
                dict(selector="th.level3.col12",    props=propriedades),
                # quarta linha - responsável
                dict(selector="th.level4.col4",     props=propriedades),
                dict(selector="th.level4.col11",     props=propriedades),
                dict(selector="th.level4.col12",     props=propriedades),
                dict(selector="th.level4.col13",     props=propriedades),
                dict(selector="th.level4.col14",     props=propriedades),
                # quinta linha - outros
                dict(selector="th.level5.col5",     props=propriedades),
                dict(selector="th.level5.col6",     props=propriedades),
                dict(selector="th.level5.col7",     props=propriedades),
                dict(selector="th.level5.col14",    props=propriedades),
                dict(selector="th.level5.col15",    props=propriedades)
            ]

df6_T.style.set_table_attributes('class="dataframe"').set_table_styles(magnify())

Nossa última análise da base PeNSE será feita em cima de um tema delicado e sensível: estupro. Filtrei jovens mulheres que já foram estupradas e separei por categoria de responsáveis pelo estupro.

Como podemos ver na tabela acima, a maioria das escolares foi estuprada pelo próprio **namorado** (78), seguido por aquelas que foram por um **familiar** (59). O estupro por um **desconhecido** é o terceiro do ranking, o que nos leva a refletir sobre a constante culpabilização da vítima, que não pode andar na rua tão tarde ou que não pode sair de casa com roupas provocantes.


---
**Fontes**:

[IPEA](https://www.ipea.gov.br/portal/index.php?id=21849&option=com_content&view=article) - Estudo analisa casos notificados de estupro

[Agência Brasil](https://agenciabrasil.ebc.com.br/direitos-humanos/noticia/2016-09/machismo-leva-culpabilizacao-da-vitima-de-violencia-sexual-diz) - Machismo leva à culpabilização da vítima de violência sexual, diz especialista






# Outras Análises

Vamos agora utilizar outras duas bases do IBGE, uma com o número de óbitos por aborto e outra com o número de nascimentos por mães jovens.

## Importando os dados

**Óbitos por Aborto**

Fonte: [Tabnet](http://tabnet.datasus.gov.br/cgi/deftohtm.exe?sim/cnv/obt10uf.def)

Para essa base, utilizei como filtro para os óbitos por aborto os CIDs abaixos:

```
CID O03 (aborto espontâneo)
CID O04 (aborto por razões médicas e legais)
CID O05 (outros tipos de aborto)
CID O06 (aborto não especificado)
CID O07 (falha de tentativa de aborto)
```

In [44]:
obitos = pd.read_csv('https://raw.githubusercontent.com/marcelagomescorrea/bootcamp_alura/main/Modulo_03/datasets/obitos_aborto.csv?raw=true',
                     encoding='ISO-8859-1', sep=';', header=0,
                     engine='python', skiprows=4, skipfooter=9)

# removendo a coluna Total
obitos.drop(['Total'], axis=1, inplace=True)
# preenchendo os nan values para zero
obitos = obitos.replace('-',0)

# removendo coluna de anos que não iremos analisar
# o ano de 2019 possui dados incompletos e por isso não será analisado
obitos.drop(['1996', '1997', '1998', '1999', '2000', '2019'], axis=1, inplace=True)

obitos['Região'].replace({
    '1 Região Norte':        'Norte',
    '2 Região Nordeste':     'Nordeste',
    '3 Região Sudeste':      'Sudeste',
    '4 Região Sul':          'Sul',
    '5 Região Centro-Oeste': 'Centro-Oeste'
}, inplace=True)

obitos.head()

,Região,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Norte,9,6,10,11,8,9,10,7,8,6,9,10,13,5,9,4,10,14
1,Nordeste,29,26,35,35,24,31,24,18,29,20,23,18,23,12,24,12,19,16
2,Sudeste,41,35,36,31,36,39,27,39,46,44,36,28,32,24,23,28,30,24
3,Sul,12,4,7,15,20,8,7,4,9,4,4,7,5,4,7,5,5,4
4,Centro-Oeste,7,7,4,7,8,5,5,8,6,5,5,6,6,10,9,7,4,10


**Nascimentos de Mães até 20 anos**

Fonte: [SIDRA](https://sidra.ibge.gov.br/tabela/2612)

Para informações de nascimentos, utilizei uma base estatística de registro civil do IBGE. Essa base (mais especificamente a tabela 2612), traz dados sobre nascidos vivos, por ano, lugar de residência e faixa etária da mãe.

In [45]:
nascimentos = pd.read_csv('https://raw.githubusercontent.com/marcelagomescorrea/bootcamp_alura/main/Modulo_03/datasets/nascimentos.csv?raw=true',
                     encoding='UTF-8', sep=';', header=0,
                     engine='python', skiprows=1, skipfooter=11)

# dropando coluna desnecessária
nascimentos.drop(['Número de nascidos por parto'], axis=1, inplace=True)

# renomeando algumas colunas
nascimentos.rename(columns={'Grande Região': 'regiao',
                   'Idade da mãe na ocasião do parto': 'faixa_etaria_mae'},
                  inplace=True)

nascimentos

,regiao,faixa_etaria_mae,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Norte,Menos de 15 anos,2420,2433,2792,2759,3015,2973,2984,2913,3028,3329,3366,3650,3586,3340,2906,2909,2838
1,Norte,15 a 19 anos,57859,56473,62120,61992,61913,61272,60094,59380,62985,64401,66271,69094,67556,62744,60472,59448,57192
2,Nordeste,Menos de 15 anos,6979,6762,7425,8009,7920,8262,8136,8117,8026,8396,8333,8522,8696,8002,7528,7270,6579
3,Nordeste,15 a 19 anos,185215,188120,196178,185874,180219,174094,164176,158238,162355,160050,157595,160534,160350,150753,146671,141313,130056
4,Sudeste,Menos de 15 anos,6759,6417,6271,6588,6814,7079,6862,6504,6488,6692,6744,7170,6594,5760,5374,4997,4419
5,Sudeste,15 a 19 anos,202906,199273,198199,190600,184415,178544,173726,167621,171027,173237,173592,173686,167189,151270,145510,134326,121620
6,Sul,Menos de 15 anos,2748,2758,2838,2873,2684,2823,2743,2571,2519,2611,2564,2756,2417,2132,1799,1641,1510
7,Sul,15 a 19 anos,70191,71006,70811,68098,63416,63373,61977,60272,60345,61199,62362,61603,59407,53456,49535,46166,42159
8,Centro-Oeste,Menos de 15 anos,1897,1815,1938,1906,1814,1964,1869,1810,1685,1740,1856,2003,1840,1695,1497,1385,1327
9,Centro-Oeste,15 a 19 anos,46080,45145,46091,43828,41318,41312,39720,38742,39278,40251,40788,41931,40313,36626,34402,32999,31643


## Gravidez na Adolescência

Vamos agora analisar o número de nascimentos por mães de até 19 anos para os últimos 19 anos no Brasil. Serão exploradas duas faixas etárias: menos de 15 anos e de 15 a 19 anos.


<img src='https://gazetadocerrado.com.br/wp-content/uploads/2017/10/embarazo_adolescentesemana_0-1200x800_c-1.jpg' width=500/>

> Fonte: Gazeta do Cerrado

In [46]:
nascimentos_melted = nascimentos.melt(id_vars=['regiao', 'faixa_etaria_mae'],
                      var_name='ano',
                      value_name='nascimentos')
nascimentos_melted.head()

,regiao,faixa_etaria_mae,ano,nascimentos
0,Norte,Menos de 15 anos,2003,2420
1,Norte,15 a 19 anos,2003,57859
2,Nordeste,Menos de 15 anos,2003,6979
3,Nordeste,15 a 19 anos,2003,185215
4,Sudeste,Menos de 15 anos,2003,6759


In [47]:
fig = make_subplots(rows=2, cols=2, x_title='Ano', y_title='Número de nascimentos',
        subplot_titles=['<b>Nordeste e Sudeste</b> - Menos de 15 anos',
                        '<b>Norte, Sul e Centro-Oeste</b> - Menos de 15 anos',
                        '<b>Nordeste e Sudeste</b> - 15 a 19 anos',
                        '<b>Norte, Sul e Centro-Oeste</b> - 15 a 19 anos'])

cores = ['red', 'forestgreen', 'dodgerblue', 'gold', 'darkorange']
regioes = nascimentos_melted['regiao'].unique()

for idx, regiao in enumerate(regioes):
    dados_regiao_ate_15 = nascimentos_melted.query(f'regiao == "{regiao}" & '\
                                    'faixa_etaria_mae == "Menos de 15 anos"')

    coluna=2
    if regiao == 'Sudeste' or regiao == 'Nordeste':
        coluna = 1

    fig.add_trace(go.Scatter(x=dados_regiao_ate_15['ano'],
                            y=dados_regiao_ate_15['nascimentos'],
                            text=dados_regiao_ate_15['nascimentos'],
                            name=regiao,
                            marker_color=cores[idx]
                ),row=1,col=coluna)

    dados_regiao_15_a_19 = nascimentos_melted.query(f'regiao == "{regiao}" & '\
                                    'faixa_etaria_mae == "15 a 19 anos"')
    fig.add_trace(go.Scatter(x=dados_regiao_15_a_19['ano'],
                            y=dados_regiao_15_a_19['nascimentos'],
                            text=dados_regiao_15_a_19['nascimentos'],
                            marker_color=cores[idx],
                            showlegend=False
                ),row=2,col=coluna)

fig.update_xaxes(dtick=1)
fig.update_yaxes(nticks=8)
fig.update_layout(
    title_text = '<b>Gravidez na Adolescência</b>: '\
                'Nascimentos ao longo dos anos por região',
    title_font = dict(size=20),
    template = 'plotly_white',
    autosize = False,
    width = 900,
    height = 600,
)

fig.show()

Para melhor visualização de possíveis tendências, separei as regiões em dois grupos por semelhança: 1) nordeste e sudeste, e 2) norte, sul e centro-oeste.

Ao analisarmos os gráficos de linha, para a faixa de 15 a 19 anos no grupo 1 (nordeste e sudeste), percebemos uma queda quase constante ao longo dos anos. Para essa mesma faixa, mas para o grupo 2 (norte, sul e centro-oeste) até há uma tendência de queda, mas bem mais sutil.

Já para a faixa de menos de 15 anos para o primeiro grupo, vemos que há um padrão de leve crescimento até pelo menos 2014-2015, seguido por uma grande queda. No grupo 2 para essa mesma faixa, o crescimento só é notado na região Norte, enquanto que nas outras regiões os números são quase constantes até 2014, após esse ano, as três regiões apresentam queda.

Realizei uma breve busca para procurar algum fato indicativo das quedas pós 2014 e não consegui achar nada relevante. Apesar disso, assusta ver que são números consideráveis de mães jovens todos os anos no Brasil.



## Óbitos por Aborto

Nossa última análise verificará como estão o número de óbitos por aborto no Brasil nos últimos 20 anos.


<img src='https://imagens.ebc.com.br/2qPCACAl4KOrAJ2AaeUhPx0UFvE=/1170x700/smart/https://agenciabrasil.ebc.com.br/sites/default/files/thumbnails/image/ffrazabr_220618313.jpg?itok=GQK4-7qA' width=600/>

> Fonte: Agência Brasil

In [48]:
obitos_melted = obitos.melt(id_vars=['Região'],
                      var_name='ano',
                      value_name='obitos')
obitos_melted.head()

,Região,ano,obitos
0,Norte,2001,9
1,Nordeste,2001,29
2,Sudeste,2001,41
3,Sul,2001,12
4,Centro-Oeste,2001,7


In [49]:
fig = make_subplots(rows=2, cols=1, x_title='Ano', y_title='Número de óbitos',
                    subplot_titles=['Sudeste e Nordeste',
                                    'Norte, Sul e Centro-Oeste'])

cores = ['red', 'forestgreen', 'dodgerblue', 'gold', 'darkorange']
regioes = obitos_melted['Região'].unique()

for idx, regiao in enumerate(regioes):
    dados_regiao = obitos_melted.query(f'Região == "{regiao}"')
    
    linha=2
    if regiao == 'Sudeste' or regiao == 'Nordeste':
        linha = 1

    fig.add_trace(go.Scatter(x=dados_regiao['ano'],
                         y=dados_regiao['obitos'],
                         text=dados_regiao['obitos'],
                         name=regiao
                  ),row=linha,col=1)

fig.update_xaxes(dtick=1)
fig.update_yaxes(nticks=10)
fig.update_layout(
    title_text = '<b>Óbitos por Aborto</b>: '\
                'Mortes por Aborto ao longo dos anos por região',
    template = 'plotly_white',
    autosize = False,
    width = 850,
    height = 600,
)

fig.show()

Nessa última análise, novamente dividi as regiões em grupos por semelhança: 1) nordeste e sudeste, e 2) norte, sul e centro-oeste.

Pelos gráficos acima, não há uma tendência muita clara de número de óbitos por aborto para as regiões. Mesmo assim, apesar dos picos e vales, o grupo 1 apresenta uma queda do número de óbitos. Para o grupo 2, os números permanecem dentro de uma faixa de valores com diversos picos e vales ao longo dos anos.

Os números acima mostram-se modestos e a explicação é simples: não há, nos sistemas de informação de saúde no Brasil, qualquer dado oficial sobre o **aborto inseguro**, gerando uma subnotificação de óbitos maternos.

Atualmente, o óbito por hemorragia puerperal (após o parto ou aborto) é a segunda maior causa direta de mortalidade materna no Brasil.


---


Fontes:

[OPAS Brasil](https://www.paho.org/bra/index.php?option=com_content&view=article&id=631) - Anahp e OPAS firmam acordo para reduzir mortalidade materna por hemorragia em hospitais privados do Brasil

[SciELO](https://www.scielo.br/scielo.php?script=sci_arttext&pid=S0102-311X2020001305001) - Aborto no Brasil: o que dizem os dados oficiais?

# Conclusão


---



![](https://imagens.brasil.elpais.com/resizer/eK1muVWje-fdrxevCxdAC0zSdXk=/1500x0/cloudfront-eu-central-1.images.arcpublishing.com/prisa/R375IGOGNGFY2J3JENA6LB7UGA.jpg)

> Fonte: El País


Em 30 de dezembro de 2020, a Argentina aprova a legalização do aborto, mas no Brasil, tal marco torna-se quase um sonho, que talvez nunca se realizará.

> “O Brasil é uma virgem que todo tarado de fora quer”

Disse o nosso presidente Jair Bolsonaro ao falar da Amazônia em julho de 2019. Um discurso carregado de misoginia que dá respaldo para que a sociedade tenha um comportamento machista, pautada por uma lógica de opressão de homens contra mulheres e de objetificação de corpos.

Crescer no Brasil não parece ser algo fácil para uma mulher, ainda mais nos tempos atuais. Temas importantes como aborto e gravidez, que deveriam ser falados nas escolas, acabam virando tabu.

Como vimos na nossa análise, nossas jovens escolares enfrentam diversos problemas: distúrbios alimentares, uso de drogas e álcool, gravidez, estupro e muitos outros. Desses, destaquei o aborto e a gravidez precoce, dois fatores bastante interligados.

O aborto hoje só é 'legalizado' para quem tem dinheiro, para quem não tem, resta apenas o aborto inseguro, clandestino, quase que de forma caseira, onde muitas perecem.

A falta de amparo da família e do governo, a ausência de estrutura emocional e financeira e o julgamento da sociedade são alguns dos motivos que fazem uma adolescente buscar o aborto como uma solução. 

Para aquelas que decidem não abortar, resta ser mãe, mas muitas delas precisam abandonar a escola, causando a longo prazo situações de vulnerabilidade social.


Apesar da queda nos números, aborto e gravidez precoce são temas que devem estar sempre em pauta.

---



Fontes:

[El País](https://brasil.elpais.com/brasil/2020-08-12/a-misoginia-do-governo-bolsonaro-vai-parar-na-justica.html) - A misoginia do Governo Bolsonaro vai parar na Justiça

[Brasil de Fato](https://www.brasildefato.com.br/2018/07/31/aborto-e-a-quarta-causa-de-morte-materna-no-brasil-afirma-pesquisadora/) - “Aborto é a quarta causa de morte materna no Brasil”, afirma pesquisadora


#**Fontes**


---


> Dados

IBGE - [Estatísticas do Registro Civil](https://sidra.ibge.gov.br/pesquisa/registro-civil/tabelas)

IBGE - [Pesquisa Nacional de Saúde do Escolar](https://www.ibge.gov.br/estatisticas/sociais/educacao/9134-pesquisa-nacional-de-saude-do-escolar.html)

Datasus/Tabnet - [Mortalidade](http://tabnet.datasus.gov.br/cgi/deftohtm.exe?sim/cnv/obt10uf.def)

---


> Artigos

[SciELO Public Health](https://www.scielo.br/scielo.php?script=sci_arttext&pid=S0102-311X2020001305001) - Aborto no Brasil: o que dizem os dados oficiais?

[SciELO Public Health](https://www.scielo.br/scielo.php?script=sci_arttext&pid=S1413-81232017000200653&lng=en&nrm=iso&tlng=pt) - Pesquisa Nacional de Aborto 2016

[SciELO Public Health](https://www.scielosp.org/article/sdeb/2019.v43n123/1132-1144/pt/) - Mortes maternas por aborto entre adolescentes no Piauí, Brasil

[Fiocruz - BVS - Saúde dos Povos Indígenas](https://ds.saudeindigena.icict.fiocruz.br/handle/bvs/632) - A gravidez na adolescência indígena: Os efeitos econômicos e sociais do comportamento sexual de risco das índias adolescentes gestantes do minicípio de Baía da Traição/PB

[PEPSIC](http://pepsic.bvsalud.org/scielo.php?script=sci_arttext&pid=S0102-73952016000100011) - Ritos de passagem: o lugar da adolescência nas sociedades indígenas Tembé Tenetehara e Kaxuyana